In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
os.chdir("D:/edwisor")

In [ ]:
data=pd.read_csv("day.csv")

In [ ]:
#DROPPING UNNECESSARY VARIABLES
data=data.drop(['instant','dteday','casual','registered'],axis=1)

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
#CONVERTING REQUIRED DATA TYPES
data['cnt']=pd.to_numeric(data['cnt'])
cnames=['season','yr','mnth','holiday','weekday','workingday','weathersit']
for i in cnames:
    data[i]=data[i].astype('object')

In [ ]:
#CHECKING MISSING VALUES
data.isnull().sum()    #no missing value found

In [ ]:
#FEATURE SELECTION

#scatter plot
plt.scatter(data['cnt'],data['temp'])

In [ ]:
plt.scatter(data['cnt'],data['atemp'])

In [ ]:
plt.scatter(data['cnt'],data['hum'])

In [ ]:
plt.scatter(data['cnt'],data['windspeed'])

In [ ]:
#correlation plot
numeric_variables=['temp','atemp','hum','windspeed','cnt']
data_corr=data.loc[:,numeric_variables]
height,width=plt.subplots(figsize=(7,5))
corr=data_corr.corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220,10,as_cmap=True),square=True,ax=width)

In [ ]:
#Dropping variable having multicollinearity problem
data=data.drop(['atemp'],axis=1)

In [ ]:
#OUTLIER ANALYSIS

In [ ]:
#creating histogram for numeric variables
plt.hist(data['temp'])

In [ ]:
plt.hist(data['hum'])

In [ ]:
plt.hist(data['windspeed'])

In [ ]:
plt.hist(data['cnt'])

In [ ]:
#creating boxplot for numerical variables
sns.boxplot(data=data['windspeed'])

In [ ]:
sns.boxplot(data=data['hum'])

In [ ]:
sns.boxplot(data=data['cnt'])

In [ ]:
sns.boxplot(data=data['temp'])

In [ ]:
#CALCUALTE INTERQUARTILE RANGE,UPPER AND LOWER FENCE FOR VARIABLE hum AND windspeed

#for variable hum
q75,q25=np.percentile(data['hum'],[75,25])
iqr=q75-q25
minimum=q25-(iqr*1.5)
maximum=q75+(iqr*1.5)

#for variable windspeed
Q75,Q25=np.percentile(data['windspeed'],[75,25])
IQR=Q75-Q25
MINIMUM=Q25-(IQR*1.5)
MAXIMUM=Q75+(IQR*1.5)

In [ ]:
#REPLACING OUTLIERS WITH NA
for i in range(len(data)):
    if(data['hum'].iloc[i] > maximum):
        data['hum'].iloc[i]=np.nan
    if(data['hum'].iloc[i] < minimum):
        data['hum'].iloc[i]=np.nan
    if(data['windspeed'].iloc[i] < MINIMUM):
        data['windspeed'].iloc[i]=np.nan
    if(data['windspeed'].iloc[i] > MAXIMUM):
        data['windspeed'].iloc[i]=np.nan

In [ ]:
#CHECKING TOTAL NUMBERS OF OUTLIERS
data.isnull().sum()

In [ ]:
#IMPUTING OUTLIERS WITH MEAN METHOD AS IT GAVE BEST RESULTS
data['windspeed']=data['windspeed'].fillna(data['windspeed'].mean())
data['hum']=data['hum'].fillna(data['hum'].mean())

In [ ]:
#CHECKING IF OUTLIERS STILL PRESENT
sns.boxplot(data['hum'])
sns.boxplot(data['windspeed'])

In [ ]:
#REMOVING OUTLIERS FROM VARIABLE windspeed AGAIN
Q75,Q25=np.percentile(data['windspeed'],[75,25])
IQR=Q75-Q25
MINIMUM=Q25-(IQR*1.5)
MAXIMUM=Q75+(IQR*1.5)
for i in range(len(data)):
    if(data['windspeed'].iloc[i] < MINIMUM):
        data['windspeed'].iloc[i]=np.nan
    if(data['windspeed'].iloc[i] > MAXIMUM):
        data['windspeed'].iloc[i]=np.nan

In [ ]:
#IMPUTING OUTLIERS WITH MEAN IN VARIABLE windspeed
data['windspeed']=data['windspeed'].fillna(data['windspeed'].mean())

In [ ]:
# AGAIN CHECKING OUTLIERS
sns.boxplot(data['windspeed'])      #no more outliers found

In [ ]:
#SELECTING SUITABLE MACHINE LEARNING ALGORITHM
training_data,test_data=train_test_split(data,test_size=0.2)
def MAPE(ya,yp):
    mape=np.mean(np.abs((ya-yp)/ya))*100
    return mape
def MAE(ya,yp):
    mae=np.mean(np.abs(ya-yp))
    return mae
def RMSE(ya,yp):
    rmse=np.sqrt(np.mean((ya-yp)**2))
    return rmse

In [ ]:
#DECISION TREE FOR REGRESSION
dt_model=DecisionTreeRegressor(max_depth=2).fit(training_data.iloc[:,0:10],training_data.iloc[:,10])
dt_predictions=dt_model.predict(test_data.iloc[:,0:10])

In [ ]:
MAPE(test_data.iloc[:,10],dt_predictions)    # mean absolute percentage error=27.96%

In [ ]:
MAE(test_data.iloc[:,10],dt_predictions)    #mean absolute error=820.257

In [ ]:
RMSE(test_data.iloc[:,10],dt_predictions)   #root mean square error=1047.757

In [ ]:
#RANDOM FOREST FOR REGRESSION
rf_model=RandomForestRegressor(n_estimators=100).fit(training_data.iloc[:,0:10],training_data.iloc[:,10])
rf_predictions=rf_model.predict(test_data.iloc[:,0:10])

In [ ]:
MAPE(test_data.iloc[:,10],rf_predictions)     #mean absolute percentage error=16.143%

In [ ]:
MAE(test_data.iloc[:,10],rf_predictions)      #mean absolute error=467.282

In [ ]:
RMSE(test_data.iloc[:,10],rf_predictions)     #root mean square error=696

In [ ]:
#if no of trees=200 ,  mape=15.78%, mae=459.7, rmse=681.66
#if no of trees=500,   mape=15.93%, mae=458.567, rmse=684.92

In [ ]:
#KNN
knn_model=KNeighborsRegressor(n_neighbors=1).fit(training_data.iloc[:,0:10],training_data.iloc[:,10])
knn_predictions=knn_model.predict(test_data.iloc[:,0:10])

In [ ]:
MAPE(test_data.iloc[:,10],knn_predictions)       #mean absolute percentage error=23.27%

In [ ]:
MAE(test_data.iloc[:,10],knn_predictions)        #mean absolute error=750.4

In [ ]:
RMSE(test_data.iloc[:,10],knn_predictions)       #rrot mean square error=1104

In [ ]:
#if nearest neighbors=3,    mape=21.52%, mae=642.11, rmse=861.77
#if nearest neighbors=5,    mape=23.78%, mae=672.7, rmse=873.58

In [ ]:
#lINEAR REGRESSION 

#converting reguiring data types
for i in range(0,training_data.shape[1]):
    training_data.iloc[:,i]=pd.to_numeric(training_data.iloc[:,i])
for i in range(0,test_data.shape[1]):
    test_data.iloc[:,i]=pd.to_numeric(test_data.iloc[:,i])

In [ ]:
lr_model=sm.OLS(training_data.iloc[:,10],training_data.iloc[:,0:10]).fit()
lr_model.summary()    #R-squared=96.8%, adjusted R-squared=96.8%

In [ ]:
lr_predictions=lr_model.predict(test_data.iloc[:,0:10])

In [ ]:
MAPE(test_data.iloc[:,10],lr_predictions)        #mean absolute percentage error = 22.8%

In [ ]:
MAE(test_data.iloc[:,10],lr_predictions)         #mean absolute error=737.56

In [ ]:
RMSE(test_data.iloc[:,10],lr_predictions)       #root mean square error=988.42